##### Configurações Iniciais

In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score, ConfusionMatrixDisplay

from Modules.operation_analiser import OperationLoader, OperationAnalyzer
from Modules.forecast_loader import ForecastLoader
from Modules.variables_info import variables_info

In [ ]:
# Parâmetros gerais
lat_porto = -25.500511
lon_porto = -48.519331
forecast_path = '../Database/forecast_data'
paradas_path = '../Database/Parada_2024_chuva.xlsx'
timebox = ('202405', '202412')

variaveis = variables_info['variaveis']
legendas = variables_info['legendas']
legendas_dict = dict(zip(variaveis, legendas))

# Repositorios de Saída
base_output = '../Output/Analise_02'
os.makedirs('../Output/Analise_02', exist_ok=True)
for subfolder in ['Tables', 'Imagens', 'Imagens/Metricas', 'Imagens/Thresholds', 'Imagens/Serie_temporais']:
    os.makedirs(os.path.join(base_output, subfolder), exist_ok=True)

##### Importação dos Dados para Analise

In [13]:
forecast = ForecastLoader(forecast_path, lat_target=lat_porto, lon_target=lon_porto)
df_forecast = forecast.load_forecasts(timebox, drop_duplicates=True)

operation_loader = OperationLoader(paradas_path)
df_paradas = operation_loader.load()

analyzer = OperationAnalyzer(df_paradas, df_forecast)
df_unificado = analyzer.marcar_paradas()

In [14]:
stats = analyzer.analisar_variaveis(variaveis, export_path='../Output/Analise_02/Tables/estatisticas_por_classe.csv')

##### Analises

###### Avaliação Individual das Variaveis para Previsão de Evento de Parada

In [15]:
resultados_thresholds = []

for var, l in zip(variaveis, legendas):
    y_true = df_unificado['parada']
    metricas_var = []

    if var.startswith('lwe_'):
        thresholds = [round(x, 2) for x in list(np.arange(0, 5.01, 0.1))]
    elif var.startswith('precipitation_probability'):
        thresholds = [round(x, 1) for x in range(0, 101, 5)]
    else:
        continue

    for thresh in thresholds:
        y_pred = (df_unificado[var] > thresh).astype(int)

        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        metricas_var.append({
            'variavel': var,
            'threshold': thresh,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })

    df_m = pd.DataFrame(metricas_var)
    df_m.to_csv(f'{base_output}/Tables/thresholds_{var}.csv', index=False)

    plt.figure(figsize=(10, 4))
    plt.plot(df_m['threshold'], df_m['f1_score'], marker='o')
    plt.title(f'{l}')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{base_output}/Imagens/Thresholds/{var}_f1_curve.png')
    plt.close()

    melhor = df_m.sort_values('f1_score', ascending=False).iloc[0].to_dict()
    resultados_thresholds.append(melhor)

best_thresholds_df = pd.DataFrame(resultados_thresholds)
best_thresholds_df = best_thresholds_df.sort_values('f1_score', ascending=False)
best_thresholds_df.to_csv(f'{base_output}/Tables/melhores_thresholds.csv', index=False)

print('Melhores thresholds por variável (baseado em F1 Score):')
display(best_thresholds_df)

Melhores thresholds por variável (baseado em F1 Score):


,variavel,threshold,precision,recall,f1_score
12,precipitation_probability_55km,25.0,0.635674,0.720430,0.675403
9,precipitation_probability_35km,25.0,0.625078,0.714695,0.666890
4,lwe_precipitation_smooth_rate_maximum_20km,0.4,0.638255,0.681720,0.659272
8,lwe_precipitation_smooth_rate_maximum_35km,0.5,0.547317,0.804301,0.651379
6,precipitation_probability_20km,15.0,0.604824,0.701075,0.649402
11,lwe_precipitation_smooth_rate_maximum_55km,1.1,0.580980,0.722581,0.644089
2,lwe_precipitation_smooth_rate_maximum_10km,0.0,0.588756,0.668100,0.625923
3,precipitation_probability_10km,0.0,0.588756,0.668100,0.625923
0,lwe_precipitation_smooth_rate,0.0,0.690710,0.453047,0.547186
1,lwe_precipitation_smooth_rate_minimum_10km,0.0,0.811441,0.274552,0.410284


###### Avaliação Combinada das Variaveis para Avaliação de Evento de Parada

In [16]:
# Parte 2: Combinação de variáveis para cenários de decisão
# Cenários: Conservador, Moderado, Arrojado

cenarios = {
    'conservador': best_thresholds_df.sort_values('recall', ascending=False).head(3),
    'moderado': best_thresholds_df.sort_values('f1_score', ascending=False).head(3),
    'arrojado': best_thresholds_df.sort_values('precision', ascending=False).head(3),
}

for nome, regras in cenarios.items():
    regras_list = [(r['variavel'], r['threshold']) for _, r in regras.iterrows()]
    condicoes = [(df_unificado[var] > thresh) for var, thresh in regras_list]
    decisao = condicoes[0]
    for c in condicoes[1:]:
        decisao = decisao & c

    df_unificado[f'cenario_{nome}'] = decisao.astype(int)

    y_true = df_unificado['parada']
    y_pred = df_unificado[f'cenario_{nome}']

    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f'\nCenário: {nome.upper()}')
    print('Variáveis usadas:')
    print(regras[['variavel', 'threshold']])
    print(f'Precision: {p:.2f} | Recall: {r:.2f} | F1 Score: {f1:.2f}')

    fig, ax = plt.subplots(figsize=(7, 5))
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax, display_labels=['Sem Parada', 'Com Parada'], cmap='Blues')
    ax.set_title(f'Matriz de Confusão - Cenário {nome.capitalize()}')
    plt.tight_layout()
    plt.ylabel('Observação')
    plt.xlabel('Modelo')
    plt.savefig(f'{base_output}/Imagens/Metricas/matriz_cenario_{nome}.png')
    plt.close()

    # Série temporal com 2 ou 4 subplots conforme o tipo de variáveis
    import matplotlib.dates as mdates

    variaveis_cenario = [v for v, _ in regras_list]  # <- correção aqui
    tipos_var = ['prob' if 'probability' in v else 'lwe' for v in variaveis_cenario]
    misto = 'prob' in tipos_var and 'lwe' in tipos_var

    def ajustar_legenda(ax):
        handles, labels = ax.get_legend_handles_labels()
        labels = [l.replace(') ', ')\n') for l in labels]
        ax.legend(handles, labels, loc='center left', bbox_to_anchor=(1.01, 0.5), fontsize='small')

    if misto:
        fig, axs = plt.subplots(4, 1, figsize=(16, 10), sharex=True)

        # Probabilidade - Parada Real
        for var in variaveis_cenario:
            if 'probability' in var:
                axs[0].plot(df_unificado['time'], df_unificado[var], label=legendas_dict.get(var, var), linewidth=1)
        axs[0].fill_between(df_unificado['time'], 0, 1, where=df_unificado['parada'] == 1, color='deepskyblue', alpha=0.15, transform=axs[0].get_xaxis_transform(), label='Parada Real')
        axs[0].set_title(f'Probabilidade - Parada Real')
        axs[0].set_ylabel('Probabilidade (%)')
        axs[0].set_ylim(bottom=0)
        axs[0].grid(True, linestyle='--', alpha=0.3)
        axs[0].set_xlim(df_unificado['time'].min(), df_unificado['time'].max())
        ajustar_legenda(axs[0])

        # Probabilidade - Sugestão Modelo
        for var in variaveis_cenario:
            if 'probability' in var:
                axs[1].plot(df_unificado['time'], df_unificado[var], label=legendas_dict.get(var, var), linewidth=1)
        axs[1].fill_between(df_unificado['time'], 0, 1, where=df_unificado[f'cenario_{nome}'] == 1, color='crimson', alpha=0.15, transform=axs[1].get_xaxis_transform(), label='Sugestão Modelo')
        axs[1].set_title(f'Probabilidade - Sugestão Modelo')
        axs[1].set_ylabel('Probabilidade (%)')
        axs[1].set_ylim(bottom=0)
        axs[1].grid(True, linestyle='--', alpha=0.3)
        ajustar_legenda(axs[1])
        axs[1].set_xlim(df_unificado['time'].min(), df_unificado['time'].max())

        # Precipitação - Parada Real
        for var in variaveis_cenario:
            if 'lwe' in var:
                axs[2].plot(df_unificado['time'], df_unificado[var], label=legendas_dict.get(var, var), linewidth=1)
        axs[2].fill_between(df_unificado['time'], 0, 1, where=df_unificado['parada'] == 1, color='deepskyblue', alpha=0.15, transform=axs[2].get_xaxis_transform(), label='Parada Real')
        axs[2].set_title(f'Precipitação - Parada Real')
        axs[2].set_ylabel('Precipitação (mm/h)')
        axs[2].set_ylim(bottom=0)
        axs[2].grid(True, linestyle='--', alpha=0.3)
        ajustar_legenda(axs[2])
        axs[2].set_xlim(df_unificado['time'].min(), df_unificado['time'].max())

        # Precipitação - Sugestão Modelo
        for var in variaveis_cenario:
            if 'lwe' in var:
                axs[3].plot(df_unificado['time'], df_unificado[var], label=legendas_dict.get(var, var), linewidth=1)
        axs[3].fill_between(df_unificado['time'], 0, 1, where=df_unificado[f'cenario_{nome}'] == 1, color='crimson', alpha=0.15, transform=axs[3].get_xaxis_transform(), label='Sugestão Modelo')
        axs[3].set_title(f'Precipitação - Sugestão Modelo')
        axs[3].set_ylabel('Precipitação (mm/h)')
        axs[3].set_ylim(bottom=0)
        axs[3].grid(True, linestyle='--', alpha=0.3)
        ajustar_legenda(axs[3])
        axs[3].set_xlim(df_unificado['time'].min(), df_unificado['time'].max())

        axs[3].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        axs[3].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.setp(axs[3].xaxis.get_majorticklabels(), rotation=45)

        fig.suptitle(f'Série Temporal - Parada Real x Decisão Modelo ({nome.capitalize()})', fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{base_output}/Imagens/Serie_temporais/serie_temporal_comparada_{nome}.png')
        plt.close()

    else:
        fig, axs = plt.subplots(2, 1, figsize=(16, 6), sharex=True)

        for var in variaveis_cenario:
            axs[0].plot(df_unificado['time'], df_unificado[var], label=legendas_dict.get(var, var), linewidth=1)
            axs[1].plot(df_unificado['time'], df_unificado[var], label=legendas_dict.get(var, var), linewidth=1)

        axs[0].fill_between(df_unificado['time'], 0, 1, where=df_unificado['parada'] == 1, color='deepskyblue', alpha=0.15, transform=axs[0].get_xaxis_transform(), label='Parada Real')
        axs[1].fill_between(df_unificado['time'], 0, 1, where=df_unificado[f'cenario_{nome}'] == 1, color='crimson', alpha=0.15, transform=axs[1].get_xaxis_transform(), label='Sugestão Modelo')

        axs[0].set_title(f'Parada Real')
        axs[1].set_title(f'Decisão Modelo')

        for ax in axs:
            ylabel = 'Probabilidade (%)' if all('probability' in v for v in variaveis_cenario) else 'Precipitação (mm/h)'
            ax.set_ylabel(ylabel)
            ax.set_ylim(bottom=0)
            ax.grid(True, linestyle='--', alpha=0.3)
            ajustar_legenda(ax)

        axs[1].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        axs[1].set_xlim(df_unificado['time'].min(), df_unificado['time'].max())
        plt.setp(axs[1].xaxis.get_majorticklabels(), rotation=45)

        fig.suptitle(f'Série Temporal - Parada Real x Decisão Modelo ({nome.capitalize()})', fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{base_output}/Imagens/Serie_temporais/serie_temporal_comparada_{nome}.png')
        plt.close()


Cenário: CONSERVADOR
Variáveis usadas:
                                      variavel  threshold
8   lwe_precipitation_smooth_rate_maximum_35km        0.5
11  lwe_precipitation_smooth_rate_maximum_55km        1.1
12              precipitation_probability_55km       25.0
Precision: 0.65 | Recall: 0.67 | F1 Score: 0.66

Cenário: MODERADO
Variáveis usadas:
                                      variavel  threshold
12              precipitation_probability_55km       25.0
9               precipitation_probability_35km       25.0
4   lwe_precipitation_smooth_rate_maximum_20km        0.4
Precision: 0.67 | Recall: 0.64 | F1 Score: 0.65

Cenário: ARROJADO
Variáveis usadas:
                                      variavel  threshold
7   lwe_precipitation_smooth_rate_minimum_35km        0.0
10  lwe_precipitation_smooth_rate_minimum_55km        0.0
5   lwe_precipitation_smooth_rate_minimum_20km        0.0
Precision: 0.91 | Recall: 0.04 | F1 Score: 0.08
